In [1]:
1+2

3

## Fine Tuning TunyLlama 1B 
Using synthetically generated data.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, TrainerCallback, LlamaTokenizer, pipeline
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

In [7]:
#cuda hai?
if torch.cuda.is_available():
    print('Hai Bhai!')
else:
    print('Nahi hai Bhai.')

Hai Bhai!


In [8]:
ds = load_dataset("aliMohammad16/einstein_answers")

README.md:   0%|          | 0.00/528 [00:00<?, ?B/s]

personaChat.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/351 [00:00<?, ? examples/s]

In [9]:
ds['train'][0]

{'prompt': '<einstein> What is time?',
 'response': 'Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference.'}

In [10]:
#v3
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [11]:
#using TinyLlama
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [77]:
tokenizer = LlamaTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")


In [67]:
tokenizer = LlamaTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizerWOYM = tokenizer

# Define special tokens
special_tokens = ["<einstein>", "<aryabhata>", "<apjkalam>", "<gandhi>", "<stevejobs>", "<cvraman>", "<user>", "<assistant>"]
tokenizerWOYM.add_special_tokens({"additional_special_tokens": special_tokens})

tokenizerWOYM.save_pretrained("fine_tuned_tokenizer")

('fine_tuned_tokenizer/tokenizer_config.json',
 'fine_tuned_tokenizer/special_tokens_map.json',
 'fine_tuned_tokenizer/tokenizer.model',
 'fine_tuned_tokenizer/added_tokens.json')

In [69]:
print(tokenizerWOYM.special_tokens_map)
print('---------------------------------')
print(tokenizerWOYM.all_special_tokens)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>', 'additional_special_tokens': ['<einstein>', '<aryabhata>', '<apjkalam>', '<gandhi>', '<stevejobs>', '<cvraman>', '<user>', '<assistant>']}
---------------------------------
['<s>', '</s>', '<unk>', '<einstein>', '<aryabhata>', '<apjkalam>', '<gandhi>', '<stevejobs>', '<cvraman>', '<user>', '<assistant>']


In [13]:
# Resize embeddings to accommodate new special tokens
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32006, 2048)

In [58]:
def format_data(example):
    bos_token = "<s>"  # Beginning of sentence
    eos_token = "</s>"  # End of sentence
    user_token = "<user>"  # Placeholder since TinyLlama doesn't define one
    assistant_token = "<assistant>"  # Placeholder for response
    
    formatted_text = f"{bos_token} {user_token} {example['prompt']} {assistant_token} {example['response']} {eos_token}"

    return {"text": formatted_text}

In [59]:
formatted_ds = ds.map(format_chat_data)

# Print an example
print(formatted_ds["train"][0]["text"])


Map:   0%|          | 0/351 [00:00<?, ? examples/s]

<s> <user> <einstein> What is time? <assistant> Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference. </s>


In [89]:
formatted_ds['train'][0]['text']

'<s> <user> <einstein> What is time? <assistant> Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference. </s>'

In [111]:
def tokeniseDataset(ds, tokenizer=tokenizer):
    
    tokenized = tokenizer(
        ds["text"], 
        padding=False,       
        truncation=True,     
        max_length=None)
    return tokenized

tokenized_dataset = formatted_ds["train"].map(tokeniseDataset, remove_columns=["text"])


Map:   0%|          | 0/351 [00:00<?, ? examples/s]

In [114]:
tokenized_dataset[0]

{'prompt': '<einstein> What is time?',
 'response': 'Time is relative. It is not an absolute entity but depends on the observer’s motion. According to my theory of relativity, time dilates for an object moving at high speeds—what you perceive as a second may not be the same for another observer in a different frame of reference.',
 'input_ids': [1,
  1,
  529,
  1792,
  29958,
  529,
  21084,
  5465,
  29958,
  1724,
  338,
  931,
  29973,
  529,
  465,
  22137,
  29958,
  5974,
  338,
  6198,
  29889,
  739,
  338,
  451,
  385,
  8380,
  7855,
  541,
  7111,
  373,
  278,
  22944,
  30010,
  29879,
  10884,
  29889,
  7579,
  304,
  590,
  6368,
  310,
  14215,
  537,
  29892,
  931,
  21749,
  1078,
  363,
  385,
  1203,
  8401,
  472,
  1880,
  961,
  5779,
  30003,
  5816,
  366,
  17189,
  573,
  408,
  263,
  1473,
  1122,
  451,
  367,
  278,
  1021,
  363,
  1790,
  22944,
  297,
  263,
  1422,
  3515,
  310,
  3407,
  29889,
  29871,
  2],
 'attention_mask': [1,
  1,
  1,
  1

In [115]:
print(tokenizer.decode(tokenizer('Hi this is not a joke')['input_ids']))

<s> Hi this is not a joke


In [121]:
print((ds['train'][39]))

{'prompt': '<einstein> What do you think about the future of space exploration?', 'response': "The future of space exploration is incredibly exciting! As we continue to push the boundaries of our knowledge, we may uncover secrets about our universe's origins and its fundamental laws. The exploration of Mars and beyond could reveal whether life exists elsewhere in the cosmos. However, we must approach this endeavor responsibly, ensuring that our actions do not harm other celestial bodies or ecosystems. As I've said, 'The important thing is not to stop questioning.' Our quest for knowledge should be guided by curiosity and respect for the universe."}


In [122]:
tokenized_dataset[39]

{'prompt': '<einstein> What do you think about the future of space exploration?',
 'response': "The future of space exploration is incredibly exciting! As we continue to push the boundaries of our knowledge, we may uncover secrets about our universe's origins and its fundamental laws. The exploration of Mars and beyond could reveal whether life exists elsewhere in the cosmos. However, we must approach this endeavor responsibly, ensuring that our actions do not harm other celestial bodies or ecosystems. As I've said, 'The important thing is not to stop questioning.' Our quest for knowledge should be guided by curiosity and respect for the universe.",
 'input_ids': [1,
  1,
  529,
  1792,
  29958,
  529,
  21084,
  5465,
  29958,
  1724,
  437,
  366,
  1348,
  1048,
  278,
  5434,
  310,
  2913,
  3902,
  12418,
  29973,
  529,
  465,
  22137,
  29958,
  450,
  5434,
  310,
  2913,
  3902,
  12418,
  338,
  29811,
  14981,
  5566,
  11407,
  29991,
  1094,
  591,
  6773,
  304,
  5503,


In [113]:
print(tokenizer.decode(tokenized_dataset[45]['input_ids']))


<s><s> <user> <einstein> What are your thoughts on creativity in science? <assistant> Creativity is essential in science! It allows us to envision new possibilities and ask questions that challenge established norms. Scientific breakthroughs often arise from imaginative thinking combined with rigorous analysis. As I once stated, 'Imagination is more important than knowledge.' The greatest scientists are those who can think outside conventional boundaries while remaining grounded in empirical evidence. </s>


### Training

In [127]:
training_args = TrainingArguments(
    output_dir="./woym",
    save_strategy="steps",
    run_name="woym",
    save_steps=500,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

<ipython-input-127-557b9e589d08>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#main man
trainer.train()

<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained("woym")
tokenizer.save_pretrained("woym")

In [ ]:
#Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()

model.push_to_hub("your_username/woym")
tokenizer.push_to_hub("your_username/woym")

In [ ]:
woym = AutoModelForCausalLM.from_pretrained("woym")
woym_tokenizer = LlamaTokenizer.from_pretrained("woym")



In [ ]:
#text generation pipeline
generator = pipeline("text-generation", model=woym, tokenizer=woym_tokenizer)

In [ ]:
# Query using special tokens
query = "<einstein> What is the nature of the universe?"
output = generator(query, max_length=100)

print(output[0]["generated_text"])

In [ ]:
def generate_answer(user_input, max_length=150, temperature=0.7, top_p=0.9, top_k=50, repetition_penalty=1.2):
    
    prompt = f"""<|user|>\n{user_input}\n<|assistant|>\n"""
    inputs = tokenizer(prompt, return_tensors="pt")

    output_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        pad_token_id=tokenizer.eos_token_id  # Proper stopping
    )

    response_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    response_text = response_text.replace(prompt, "").strip()  # Remove prompt from output

    return response_text
